TO DO

- is it possible to add captaincy bonus?
- add an optional constraint to allow only one defender from any given team (risk mitigation)?

In [1]:
import pandas as pd
import numpy as np
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable

In [2]:
# function to solve the optimization problem
def optimize(df, max_price, expected_column):
    
    # PRELIMINARIES
    
    # Create the model
    model = LpProblem(name="FPL", sense=LpMaximize)    
    variables = [LpVariable(name=f'{ix}', cat='Binary') for ix in df.index]
    prices = [df.loc[ix,'now_cost']/10.0 for ix in df.index]
    # measure of player quality
    expected_points = [df.loc[ix,expected_column] for ix in df.index]
    goalkeepers = [1.0 if df.loc[ix,'element_type']==1 else 0.0 for ix in df.index]
    defenders = [1.0 if df.loc[ix,'element_type']==2 else 0.0 for ix in df.index]
    midfielders = [1.0 if df.loc[ix,'element_type']==3 else 0.0 for ix in df.index]
    forwards = [1.0 if df.loc[ix,'element_type']==4 else 0.0 for ix in df.index]
    teams = []
    for team in df['team'].unique():
        teams.append( [1.0 if df.loc[ix,'team']==team else 0.0 for ix in df.index] )

    # CONSTRAINTS

    # select 11 players
    model += lpSum(variables) == 11
    # set maximum price for starting 11
    model += np.dot(prices,variables) <= max_price
    # only 1 goalkeeper
    model += np.dot(goalkeepers,variables) == 1
    # at least 3 defenders
    model += np.dot(defenders,variables) >= 3
    # at most 5 defenders
    model += np.dot(defenders,variables) <= 5
    # at most 5 midfielders
    model += np.dot(midfielders,variables) <= 5
    # at least 1 forward
    model += np.dot(forwards,variables) >= 1
    # at most 3 forwards
    model += np.dot(forwards,variables) <= 3
    # max 3 players from any given team
    for team in teams:
        model += np.dot(team,variables) <= 3
    
    # OBJECTIVE
    # if possible, SHOULD ADD CAPTAIN'S DOUBLE POINTS TO THE OBJECTIVE
    model += np.dot(expected_points,variables)

    # SOLVE OPTIMIZATION
    
    status = model.solve()
    print(f'Status: {LpStatus[model.status]}')
    #print(f'Mean total points per gameweek: {model.objective.value()}')

    players = [int(str(var)) for var in model.variables() if var.value()==1]
    dream_team = df.loc[players]
    cost = (dream_team['now_cost']/10.0).sum()
    exp_points = dream_team[expected_column].sum() + dream_team[expected_column].max()
    print(f'Cost: {cost}')
    if expected_column == 'adjusted points per game':
        print(f'Expected points per week: {exp_points}')
    elif expected_column == 'xPoints_next10':
        exp_points = exp_points / 10
        print(f'Expected points per week: {exp_points}')
    else:
        print(f'Expected points: {exp_points}')
    display(dream_team[['element_type','web_name','now_cost','games played',expected_column]].sort_values('element_type'))

In [3]:
gameweek = 23
# fetch FPL data
filepath = '../data/fpl/data_week' + str(gameweek) + '.csv'
df = pd.read_csv(filepath, index_col=0)
df

,adjusted points,adjusted points per game,assists,assists_week1,assists_week10,assists_week11,assists_week12,assists_week13,assists_week14,assists_week15,...,xPoints week 3,xPoints week 4,xPoints week 5,xPoints week 6,xPoints week 7,xPoints week 8,xPoints week 9,xPoints_next10,yellow_cards,gameweek 33 prediction
id,,,,,,,,,,,,,,,,,,,,,
1,7.044931,2.348310,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1.089483,NaN,NaN,NaN,NaN,NaN,NaN,37.406294,0,4.076060
2,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.907182,0,3.230990
3,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.424405,0,2.240415
4,50.411000,3.600786,1,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000,10.5,3.6,3.8,2.4,6.8,4.071,25.663554,3,2.681040
5,6.359907,1.589977,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.910517,1.0,NaN,NaN,NaN,NaN,NaN,19.404380,1,1.884078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
661,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.168127,0,1.900161
685,5.201118,2.600559,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.823735,1,1.816863
686,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.987013,0,1.847196


In [4]:
# only consider players that have played at least some minimum number of games
df = df[df['games played']>=3]
df

,adjusted points,adjusted points per game,assists,assists_week1,assists_week10,assists_week11,assists_week12,assists_week13,assists_week14,assists_week15,...,xPoints week 3,xPoints week 4,xPoints week 5,xPoints week 6,xPoints week 7,xPoints week 8,xPoints week 9,xPoints_next10,yellow_cards,gameweek 33 prediction
id,,,,,,,,,,,,,,,,,,,,,
1,7.044931,2.348310,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1.089483,NaN,NaN,NaN,NaN,NaN,NaN,37.406294,0,4.076060
4,50.411000,3.600786,1,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000,10.500000,3.600000,3.800000,2.400000,6.800000,4.071000,25.663554,3,2.681040
5,6.359907,1.589977,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.910517,1.000000,NaN,NaN,NaN,NaN,NaN,19.404380,1,1.884078
6,49.553000,3.097062,3,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000,NaN,NaN,NaN,1.000000,3.600000,2.000000,35.112450,0,3.504342
7,19.908442,1.990844,1,0.0,NaN,NaN,NaN,NaN,NaN,0.0,...,-2.000000,NaN,NaN,2.332871,NaN,NaN,NaN,19.454559,4,2.072402
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,13.100000,1.190909,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,...,1.000000,NaN,1.000000,NaN,NaN,1.300000,NaN,19.151312,0,1.479033
461,49.377375,2.904551,0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,...,5.548812,4.418731,2.382085,NaN,5.048812,NaN,NaN,28.387634,0,2.138596
470,43.919148,3.992650,3,NaN,1.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,3.861196,35.083746,1,2.469736


In [5]:
optimize(df, 85.2, f'gameweek {gameweek+1} prediction')

Status: Optimal
Cost: 85.00000000000001
Expected points: 59.9361138343811


,element_type,web_name,now_cost,games played,gameweek 24 prediction
id,,,,,
156,1,Butland,44,3.0,4.672294
237,2,Alexander-Arnold,85,20.0,4.863126
256,2,Cancelo,71,22.0,4.642756
370,2,Reguilón,54,18.0,4.537735
144,3,Gallagher,62,19.0,4.627248
233,3,Salah,127,20.0,5.608174
240,3,Jota,84,21.0,5.148986
359,3,Son,106,17.0,5.166996
419,3,Benrahma,59,20.0,4.753629


In [7]:
optimize(df, 86, 'adjusted points per game')

Status: Optimal
Cost: 84.6
Expected points per week: 73.8289880488729


,element_type,web_name,now_cost,games played,adjusted points per game
id,,,,,
156,1,Butland,44,3.0,5.828420
135,2,Chilwell,56,6.0,6.204582
142,2,James,62,16.0,5.108686
237,2,Alexander-Arnold,85,20.0,6.616628
256,2,Cancelo,71,22.0,5.843542
233,3,Salah,127,20.0,7.660032
265,3,Foden,80,15.0,5.644349
268,3,Torres,67,4.0,6.316499
359,3,Son,106,17.0,5.863891


In [8]:
df = df.drop([135, 268, 156])
optimize(df, 85.2, 'adjusted points per game')

Status: Optimal
Cost: 85.2
Expected points per week: 69.45990840342142


,element_type,web_name,now_cost,games played,adjusted points per game
id,,,,,
333,1,Forster,44,5.0,4.146850
142,2,James,62,16.0,5.108686
234,2,Robertson,71,16.0,5.041628
237,2,Alexander-Arnold,85,20.0,6.616628
256,2,Cancelo,71,22.0,5.843542
259,2,Laporte,58,19.0,4.791944
233,3,Salah,127,20.0,7.660032
265,3,Foden,80,15.0,5.644349
359,3,Son,106,17.0,5.863891


In [ ]:
df = df.drop([333])
optimize(df, 85.2, 'adjusted points per game')

In [ ]:
df = df.drop([129])
optimize(df, 85.2, 'adjusted points per game')

In [ ]:
optimize(df, 85.2, 'xPoints_next10')